In [1]:
from math import log
from collections import defaultdict
import re #regular expression
import math
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
#import nltk(natural language tokenizer) - konlpy
%matplotlib inline

목표 : 메일이 스팸인지 아닌지 구분하는 분류기 만들기.

In [3]:
df = pd.read_csv('spam.csv',encoding = "ISO-8859-1")
df.dropna(inplace=True, axis=1)
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df['is_spam'] = df.v1.map(lambda x: 1 if x =='spam' else 0) #lambda 함수 : if문 1줄컷 getdummyes() 
df.head()

subset = df[['v2', 'is_spam']]
tuples = [tuple(x) for x in subset.values]
data = tuples

lambda() 함수 : dummy변수 코딩용.

In [5]:
subset.head()

,v2,is_spam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [6]:
subset.describe()

,is_spam
count,5572.000000
mean,0.134063
std,0.340751
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [7]:
def split_data(data, prob):
    """split data into fractions [prob, 1 - prob]"""
    results = [], []
    for row in data:
        results[0 if random.random() < prob else 1].append(row)
    return results

random.seed(1131) # just so you get the same answers as me
train_data, test_data = split_data(data, 0.75)

[0 if ...] 구문

In [8]:
## 단어 전처리
def tokenize(message):
    message = message.lower() # convert to lowercase
    all_words = re.findall("[a-z0-9']+", message) # extract the words
    return set(all_words) # remove duplicates

tokenization : 문장을 단어의 의미별로 토큰화.

기본 : 띄어쓰기 단위로 의미 추출. 이후 stopwords(불용어) 처리.

stopwords : the, an, a, '은', '는', ...

이외 사전 : https://github.com/entelecheia/eKoNLPy 등.

In [9]:
# 스팸인지 아닌지 담아주자
# word = [spam, non_spam]
def count_words(training_set):
    """training set consists of pairs (message, is_spam)"""
    counts = defaultdict(lambda: [0, 0]) #dictionary를 만드는 함수. 키 값이 없더라도 기본값 정의를 통해 에러를 도출하지 않게 설정
    for message, is_spam in training_set:
        for word in tokenize(message):
            counts[word][0 if is_spam else 1] += 1
    return counts

In [10]:
count_words(train_data)['www']

[66, 2]

In [11]:
count_words(train_data)['latest']

[24, 1]

In [12]:
count_words(train_data)['lol']

[0, 57]

In [13]:
count_words(train_data)['free']

[111, 48]

### 확률 계산하기
prob_if_spam =$ p(X_1 =1 |spam)$ = '당첨'단어가 들어간 메일 중 스팸의 개수 / 전체 스팸 메일 수 = word count 함수의 spam_count/ total_spams

$ p(X_1 =1 |ham)$ = word count 함수의 ham_count/ total_non_spams

In [11]:
def word_probabilities(counts, total_spams, total_non_spams, k=0.5):
    """turn the word_counts into a list of triplets
    w, p(w | spam) and p(w | ~spam)"""
    return [(w,
            (spam + k) / (total_spams + 2 * k),
            (non_spam + k) / (total_non_spams + 2 * k))
            for w, (spam, non_spam) in counts.items()]

k는 왜 더해졌나? 'lol'에서 spam이 0회였으니 아무리 안 봤어도 최소의 가중치 k를 부여하는 것(laplas smoothing)

 $p(c)$ 와 $p(\sim c)$를 구하여 $ p(c|x) = \frac{p(x|c)p(c) } {p(x|c) p(c) + p(x|\sim c)*p(\sim c)}$ 를 계산한다.

** p(c), p(~c) 구하기 **

In [14]:
word_probabilities(count_words(train_data), prob, )

TypeError: word_probabilities() missing 2 required positional arguments: 'total_spams' and 'total_non_spams'

In [15]:
count_words(train_data)

defaultdict(<function __main__.count_words.<locals>.<lambda>()>,
            {'2005': [3, 0],
             'text': [78, 48],
             'comp': [9, 2],
             'apply': [21, 2],
             't': [42, 26],
             'final': [11, 2],
             'in': [52, 547],
             "c's": [10, 0],
             'entry': [15, 0],
             'fa': [2, 0],
             "08452810075over18's": [2, 0],
             'may': [4, 27],
             'tkts': [4, 0],
             'rate': [22, 2],
             'wkly': [11, 0],
             'to': [319, 922],
             'txt': [101, 10],
             'free': [111, 48],
             'win': [43, 6],
             '2': [120, 198],
             'a': [212, 648],
             'receive': [24, 5],
             '21st': [2, 0],
             'cup': [5, 2],
             '87121': [4, 0],
             'std': [11, 0],
             'question': [6, 13],
             'dun': [0, 42],
             'so': [18, 303],
             'u': [96, 518],
             'c': [22, 

In [13]:
df_is_spam = word_probabilities(, count_words(train_data))

prob_spam = df_is_spam.mean() #p(c)
prob_n_spam = -1 - prob_spam #p(~c)
log_prob_spam = math.log(prob_spam)
log_prob_n_spam = math.log(prob_n_spam)

SyntaxError: invalid syntax (3335017334.py, line 1)

In [37]:
prob_n_spam

NameError: name 'prob_n_spam' is not defined

In [38]:
def spam_probability(word_probs, message):
    message_words = tokenize(message)
    log_prob_if_spam = log_prob_if_not_spam = 0.0
    # iterate through each word in our vocabulary
    for word, prob_if_spam, prob_if_not_spam in word_probs:
    # if *word* appears in the message,
    # add the log probability of seeing it
        if word in message_words:
            log_prob_if_spam += math.log(prob_if_spam)
            log_prob_if_not_spam += math.log(prob_if_not_spam)
        # if *word* doesn't appear in the message
        # add the log probability of _not_ seeing it
        # which is log(1 - probability of seeing it)
        else:
            log_prob_if_spam += math.log(1.0 - prob_if_spam)
            log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)
    prob_if_spam = math.exp(log_prob_if_spam + log_prob_spam) 
    prob_if_not_spam = math.exp(log_prob_if_not_spam + log_prob_n_spam) 
    return prob_if_spam / (prob_if_spam + prob_if_not_spam) ,  prob_if_not_spam / (prob_if_spam + prob_if_not_spam)
    #return prob_if_spam,  prob_if_not_spam  #p(x)제외

#### 이제 Naive Bayes Classifier로 모든걸 집어넣자

In [40]:
class NaiveBayesClassifier:
    def __init__(self, k = 0.5):
        self.k = k
        self.word_probs = []
    def train(self, training_set):
        # count spam and non-spam messages
        num_spams = len([is_spam
                        for message, is_spam in training_set
                        if is_spam])
        num_non_spams = len(training_set) - num_spams
        # run training data through our "pipeline"
        word_counts = count_words(training_set)
        self.word_probs = word_probabilities(word_counts,
                                            num_spams,
                                            num_non_spams,
                                            self.k)
    def classify(self, message):
        return spam_probability(self.word_probs, message)

In [41]:
classifier = NaiveBayesClassifier()
classifier.train(train_data)

In [43]:
# triplets (subject, actual is_spam, predicted spam probability)
classified = [(subject, is_spam, classifier.classify(subject))
              for subject, is_spam in test_data]

NameError: name 'log_prob_spam' is not defined

In [44]:
len(classified)

NameError: name 'classified' is not defined

In [46]:
classified[0:30] #message, true y, prob(p(spam|x), p(ham|x))

NameError: name 'classified' is not defined